### 조기 종료(early stopping)

In [1]:
import pandas as pd
df = pd.read_csv("../../data/classification/winequality-red.csv")
X = df.drop('y', axis = 1)
y = df['y']

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.model_selection import ParameterGrid, KFold
from sklearn.metrics import accuracy_score

grid = ParameterGrid(
        {"hidden_layer_sizes": [(10, ), (10, 10, 10), (20, 20), (30, 30, 30)],
        "early_stopping": [True, False],
        "max_iter": [100, 500, 1000],
        "random_state":[2022]})

In [4]:
import time
kf = KFold(n_splits = 5, shuffle = True, random_state = 2022)
start_time = time.time()
best_score = -1
for parameter in grid:
    score = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model = MLP(**parameter).fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score += accuracy_score(y_test, y_pred) / 5
    if score > best_score:
        best_score = score
        best_parameter = parameter
    
end_time = time.time()
print(best_parameter, best_score)
print(int(end_time - start_time))

{'early_stopping': False, 'hidden_layer_sizes': (10, 10, 10), 'max_iter': 1000, 'random_state': 2022} 0.5885050940438872
89


In [6]:
start_time = time.time()
best_score = -1
for parameter in grid:
    score = 0
    k = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model = MLP(**parameter).fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score += accuracy_score(y_test, y_pred) / 5
        k += 1
        if score * (5/k) < best_score * 0.95:
            break
        
    if score > best_score:
        best_score = score
        best_parameter = parameter
    
end_time = time.time()
print(best_parameter, best_score)
print(int(end_time - start_time))

{'early_stopping': False, 'hidden_layer_sizes': (10, 10, 10), 'max_iter': 1000, 'random_state': 2022} 0.5885050940438872
81


In [18]:
import numpy as np
start_time = time.time()
best_score = -1
for parameter in grid:
    score_list = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model = MLP(**parameter).fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        score_list.append(score)
        if score < best_score * 0.95:
            break
    
    mean_score = np.mean(score_list)
    if mean_score > best_score:
        best_score = mean_score
        best_parameter = parameter
    
end_time = time.time()
print(best_parameter, best_score)
print(int(end_time - start_time))

{'early_stopping': False, 'hidden_layer_sizes': (10, 10, 10), 'max_iter': 1000, 'random_state': 2022} 0.5885050940438872
76


### 다중 충실도(multi-fidelity)

In [7]:
from scipy.stats import rankdata
start_time = time.time()

s_param_idx_list = np.arange(len(grid)).astype(int)
for i in range(5):
    score_list = []
    for s_param_idx in s_param_idx_list:
        s_parameter = grid[s_param_idx]
        s_idx = np.random.choice(X.index,
                                 int(len(X) / len(s_param_idx_list)),
                                 replace = False)
        
        s_X = X.loc[s_idx]
        s_y = y.loc[s_idx]
        
        score = 0
        for train_index, test_index in kf.split(s_X):
            s_X_train, s_X_test = s_X.iloc[train_index], s_X.iloc[test_index]
            s_y_train, s_y_test = s_y.iloc[train_index], s_y.iloc[test_index]
            model = MLP(**s_parameter).fit(s_X_train, s_y_train)
            s_y_pred = model.predict(s_X_test)
            score += accuracy_score(s_y_test, s_y_pred) / 5
        score_list.append(score)
    score_list = np.array(score_list)
    s_param_idx_list = rankdata(-score_list)[:int(len(s_param_idx_list) * 0.8)] - 1
    s_param_idx_list = s_param_idx_list.astype(int)

end_time = time.time()
print(best_parameter, best_score)
print(int(end_time - start_time))

{'early_stopping': False, 'hidden_layer_sizes': (10, 10, 10), 'max_iter': 1000, 'random_state': 2022} 0.5885050940438872
43
